Show all columns when showing dataframe

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

Dataset path

In [3]:
train_feature_path = "./Dataset/training_set_features.csv"
train_label_path = "./Dataset/training_set_labels.csv"
test_path = "./Dataset/test_set_features.csv"
submission_path = "./Dataset/submission_format.csv"

Import train data

In [4]:
x_data = pd.read_csv(train_feature_path, index_col='respondent_id')
y_data = pd.read_csv(train_label_path, index_col='respondent_id')

Create a backup of train data

In [5]:
x_data_backup = x_data.copy()
y_data_backup = y_data.copy()

Restore train data from backup data

In [6]:
x_data = x_data_backup.copy()
y_data = y_data_backup.copy()

Some samples in train data

In [7]:
x_data

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,1.0,1.0,5.0,2.0,2.0,65+ Years,Some College,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,2.0,2.0,5.0,1.0,1.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,4.0,2.0,5.0,4.0,2.0,55 - 64 Years,Some College,White,Female,NaN,Not Married,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN


In [ ]:
y_data

Information about train data (column name, data type, non-null values number)

In [ ]:
x_data.info()

In [ ]:
y_data.info()

Lists of values of each column

In [ ]:
for col in x_data.columns:
    print(x_data[col].value_counts())
    print()

In [ ]:
for col in y_data.columns:
    print(y_data[col].value_counts())
    print()

Numbers of missing values in each column

In [ ]:
missValData = []

for column in x_data.columns:
    missValData.append([column, len(x_data) - x_data[column].count()])

missValData = pd.DataFrame(missValData, columns=['column_name', 'missing_value_count'])
missValData

Split dataset into train, test

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
RANDOM_SEED = 6

x_train, x_test, y_train, y_test = train_test_split(
    x_data,
    y_data,
    test_size=0.3,
    shuffle=True,
    stratify=y_data,
    random_state=RANDOM_SEED
)

Split features into categorical and numeric

In [10]:
cat_cols = list(x_data.select_dtypes('object').columns)
num_cols = list(x_data.select_dtypes('number').columns)

Preprocessing

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [23]:
cat_pl= Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value="missing")),
        ('onehot', OneHotEncoder())
    ]
)

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
num_pl = Pipeline(
    steps=[
           ('imputer', SimpleImputer(strategy='constant', fill_value=-1)),
           ('scaler', StandardScaler())
    ]
)

In [15]:
from sklearn.compose import ColumnTransformer

In [24]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pl, num_cols),
        ('cat', cat_pl, cat_cols)
    ]
)

Complete pipeline

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

In [18]:
estimators = MultiOutputClassifier(
    estimator=LogisticRegression(penalty="l2", C=1)
)

In [27]:
completed_pl = Pipeline(
    steps=[
            ("preprocessor", preprocessor), 
            ("estimator", estimators)
    ]
)

In [28]:
completed_pl.fit(x_train, y_train)

d:\Coding\Flu-Shot-Learning---Predict-H1N1-and-Seasonal-Flu-Vaccines\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Coding\Flu-Shot-Learning---Predict-H1N1-and-Seasonal-Flu-Vaccines\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=-1,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['h1n1_concern',
                                                   'h1n1_knowledge',
                                                   'behavioral_antiviral_meds',
                                                   'behavioral_avoidance',
                                                   'behavioral_face_mask',
                                                   'behavioral_wash_hands',
                                                   'behavioral_large_gatherings',
                                                   'behavioral...
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['age_group', 'education',
                                                   'race', 'sex',
                                                   'income_poverty',
                                                   'marital_status',
                                                   'rent_or_own',
                                                   'employment_status',
                                                   'hhs_geo_region',
                                                   'census_msa',
                                                   'employment_industry',
                                                   'employment_occupation'])])),
                ('estimator',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1)))])

In [29]:
preds = completed_pl.predict_proba(x_test)

In [30]:
y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": preds[0][:, 1],
        "seasonal_vaccine": preds[1][:, 1],
    },
    index = y_test.index
)

In [31]:
from sklearn.metrics import roc_auc_score

In [32]:
roc_auc_score(y_test, y_preds)

0.8477912091388933

(Function) Check if values in a column is numeric

In [ ]:
def isNumeric(data: pd.Series):
    return data.dtype != "object"

(Function) Classify columns into 2 types: numeric and non-numeric

In [ ]:
def split_Num_NNum(ds: pd.DataFrame):
    num = []
    nnum = []
    
    for col in ds.columns:
        if isNumeric(ds[col]):
            num.append(col)
        else:
            nnum.append(col)

    return num, nnum

Classify columns into 2 types: numeric and non-numeric

In [ ]:
num, nnum = split_Num_NNum(train_data)

(Function) Convert non-numeric values into numeric ones

In [ ]:
def convertToNum(data: pd.DataFrame, colList: list):
    for col in colList:
        value_list = data[col].value_counts().index.sort_values(ascending=True)
        for i in range(len(value_list)):
            data[col].replace(to_replace=value_list[i], value=i, inplace=True)

In [ ]:
convertTable = {}
for col in x_data.select_dtypes('object').columns:
    convertTable.update({col: x_data[col].value_counts().index.sort_values(ascending=True)}) 

In [ ]:
convertToNum(train_data, nnum)

Replace all NaN values by -1

In [ ]:
import numpy as np

In [ ]:
train_data.replace(to_replace=np.nan, value=-1, inplace=True)

In [ ]:
def preprocessing(data: pd.DataFrame):
    nnum = split_Num_NNum(data)[1]
    convertToNum(data, nnum)
    data.replace(to_replace=np.nan, value=-1, inplace=True)

In [ ]:
train_data

In [ ]:
preprocessing(train_data)

Find features that affect to dependent variables

In [ ]:
from scipy import stats as st

In [ ]:
num.remove('h1n1_vaccine')
num.remove('seasonal_vaccine')
num.remove('respondent_id')

In [ ]:
def checkDisCorel(x: str, y: str, data: pd.DataFrame, std_pval = 0.05):
    data_test = data[[x, y]]
    data_test.dropna(inplace=True)
    data_grp = data_test.groupby([x])

    ind_grp = []
    
    for val in data_grp[x].value_counts().index:  
        ind_grp.append(data_grp.get_group(val)[y])

    score, pval = st.f_oneway(*ind_grp)

    isCorel = False
    if pval < std_pval:
        isCorel = True

    return [x, y, score, pval, isCorel]

In [ ]:
corelData1 = []
corelData2 = []

for col in num:
    corelData1.append(checkDisCorel(col, 'h1n1_vaccine', train_data))
    corelData2.append(checkDisCorel(col, 'seasonal_vaccine', train_data))

for col in nnum:
    corelData1.append(checkDisCorel(col, 'h1n1_vaccine', train_data))
    corelData2.append(checkDisCorel(col, 'seasonal_vaccine', train_data))

In [ ]:
def getIndFeatures(data: pd.DataFrame):
    col_list = list(data.columns)
    col_list.remove('respondent_id')
    col_list.remove('h1n1_vaccine')
    col_list.remove('seasonal_vaccine')

    return col_list

In [ ]:
def checkCorel(data: pd.DataFrame, target: str):
    col_list = getIndFeatures(data)

    corelData = []

    for col in col_list:
        corelData.append(checkDisCorel(col, target, train_data))

    return pd.DataFrame(corelData, columns=['x', 'y', 'score', 'pval', 'isCorel'])\
            .sort_values(by='pval', ascending=True)\
            .loc[corelData1['isCorel']]['x']

In [ ]:
corelData1 = pd.DataFrame(corelData1, columns=['x', 'y', 'score', 'pval', 'isCorel'])\
            .sort_values(by='pval', ascending=True)

In [ ]:
corelData2 = pd.DataFrame(corelData2, columns=['x', 'y', 'score', 'pval', 'isCorel'])\
            .sort_values(by='pval', ascending=True)

In [ ]:
h1n1_corel = corelData1.loc[corelData1['isCorel']]['x']
seasonal_corel = corelData2.loc[corelData2['isCorel']]['x']

Split dataset into train, test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y_data = train_data[['h1n1_vaccine', 'seasonal_vaccine']]
# x_data = train_data.copy()
# x_data = train_data.drop(['h1n1_vaccine', 'seasonal_vaccine'], axis=1)
x_data = train_data[h1n1_corel]

In [ ]:
RANDOM_SEED = 6

x_train, x_eval, y_train, y_eval = train_test_split(
    x_data,
    y_data,
    test_size=0.3,
    shuffle=True,
    stratify=y_data,
    random_state=RANDOM_SEED
)

Build the model

In [ ]:
from sklearn.preprocessing import StandardScaler
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
# from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline


from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
preprocessor = Pipeline([
    ('standard_scaler', StandardScaler())
])

In [ ]:
estimators = LogisticRegression(penalty="l2", C=1)

In [ ]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators)
])

In [ ]:
# %%time

# Train model
full_pipeline.fit(x_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds = full_pipeline.predict_proba(x_eval)

(Function) Proposed system

In [ ]:
def build_model(x_data: pd.DataFrame, y_data: pd.DataFrame):
    train_features = {}
    for col in y_train.columns:
        train_features.update({col: findTrainFeatures(x_data, y_data[col])})

    wrangling_data(x_data, y_data)

    RANDOM_SEED = 6
    x_train, x_eval, y_train, y_eval = train_test_split(
        x_data,
        y_data,
        test_size=0.3,
        shuffle=True,
        stratify=y_data,
        random_state=RANDOM_SEED
    )

    model = {}
    for col in y_train.columns:
        model.update({col: training(x_train[train_features[col]], y_train[col])})

    return model
    

In [ ]:
y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": preds[0][:, 1],
        "seasonal_vaccine": preds[1][:, 1],
    },
    index = y_eval.index
)
# print("y_preds.shape:", y_preds.shape)
# y_preds.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_roc(y_true, y_score, label_name, ax):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    ax.plot(fpr, tpr)
    ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
    ax.set_ylabel('TPR')
    ax.set_xlabel('FPR')
    ax.set_title(
        f"{label_name}: AUC = {roc_auc_score(y_true, y_score):.4f}"
    )

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7, 3.5))

plot_roc(
    y_eval['h1n1_vaccine'], 
    y_preds['h1n1_vaccine'], 
    'h1n1_vaccine',
    ax=ax[0]
)
plot_roc(
    y_eval['seasonal_vaccine'], 
    y_preds['seasonal_vaccine'], 
    'seasonal_vaccine',
    ax=ax[1]
)
fig.tight_layout()

In [ ]:
roc_auc_score(y_eval, y_preds)

In [ ]:
test_features_df = pd.read_csv(test_path, index_col="respondent_id")

In [ ]:
preprocessing(test_features_df)
test_probas = full_pipeline.predict_proba(test_features_df[h1n1_corel])

In [ ]:
submission_df = pd.read_csv(submission_path, index_col="respondent_id")

In [ ]:
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

In [ ]:
submission_df["h1n1_vaccine"] = test_probas[0][:, 1]
submission_df["seasonal_vaccine"] = test_probas[1][:, 1]

In [ ]:
submission_df.to_csv('my_submission.csv', index=True)